# UEC実践ソフトウェア開発基礎論2023-レポート課題2
田中久温

## 概要
元コンペ: https://www.kaggle.com/competitions/bnp-paribas-cardif-claims-management \
個人保険のデータセット．顧客から来た保険金の請求を速やかにチェックして承認 or 非承認しないといけない．\
データセットに含まれる請求には2つのカテゴリが含まれていて，どっちのカテゴリか判定したい．\
出力は確率，評価指標は logloss \
目的変数も説明変数も匿名で，特に説明無し．順序変数はないらしい．

## 手法
この notebook では，RandomForest を試す．\
この notebook は4つ目に実装する．XGBoost のコードをベースにやる．

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
train_df = pd.read_csv("../input/bnp-paribas-cardif-claims-management/train.csv.zip")
test_df = pd.read_csv("../input/bnp-paribas-cardif-claims-management/test.csv.zip")
sample_df = pd.read_csv("../input/bnp-paribas-cardif-claims-management/sample_submission.csv.zip")

# EDA は省略

# 特徴量作成 XGBoost と一緒

In [ ]:
train_test_df = pd.concat([train_df, test_df])
categorical_columns = [c for c in train_test_df.columns if train_test_df[c].dtype == 'O'] # 先生のサンプルから
numeric_columns = [c for c in train_test_df.columns[2:] if c not in categorical_columns]

In [ ]:
# numeric
for c in numeric_columns:
    nan_ratio = train_test_df[c].isna().mean()

    if nan_ratio < 0.1:
        # nan が 10% 未満なら平均値で埋める．(あんまり考慮したくない)
        column_mean = train_test_df[c].mean()
        train_test_df[c] = train_test_df[c].fillna(column_mean)
    else:
        # nan が 10% 以上あれば -999 で埋める．(モデルが考慮できるようにする)
        train_test_df[c] = train_test_df[c].fillna(-999)
    
# categorical
for c in categorical_columns:
    nan_count = train_test_df[c].isna().sum()

    if nan_count <= 10:
        # nan が 10個以下なら，最頻値で埋める．(あんまり考慮したくない)
        train_test_df[c] = train_test_df[c].fillna(train_test_df[c].mode()[0])
    else:
        # nan が 10個より多いなら，カテゴリとして扱う．(モデルが考慮できるようにする)
        train_test_df[c] = train_test_df[c].fillna(train_test_df[c].mode()[0])


from sklearn.preprocessing import LabelEncoder

for c in categorical_columns:
    le = LabelEncoder()
    train_test_df[c] = le.fit_transform(train_test_df[c])

In [ ]:
use_cols = numeric_columns + categorical_columns

# 最初の学習

In [ ]:
# データ分割
from sklearn.model_selection import train_test_split 

train_ft_df = train_test_df[train_test_df['target'].notna()][use_cols]
train_y_df = train_test_df[train_test_df['target'].notna()]['target'].astype(int)
test_ft_df = train_test_df[train_test_df['target'].isna()][use_cols]

train_X, valid_X, train_y, valid_y = train_test_split(train_ft_df, train_y_df, test_size=0.2, shuffle=True)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = RandomForestClassifier(n_estimators=20, criterion='log_loss')
clf.fit(train_X, train_y)
clf.score(valid_X, valid_y)

In [ ]:
pred_test = clf.predict_proba(test_ft_df)[:, 1]
sub_df = pd.DataFrame({
    "ID": test_df["ID"], 
    "PredictedProb": pred_test
}).to_csv("tanaka_sub_randomf.csv", index=False)

public 0.606

結構学習に時間かかる割に酷い精度...?

ちゃんとハイパラチューニングするってなるとかなり大変な気がする．

いったん100で回す．

In [ ]:
clf = RandomForestClassifier(n_estimators=100, criterion='log_loss')
clf.fit(train_X, train_y)
clf.score(valid_X, valid_y)

In [ ]:
pred_test = clf.predict_proba(test_ft_df)[:, 1]
sub_df = pd.DataFrame({
    "ID": test_df["ID"], 
    "PredictedProb": pred_test
}).to_csv("tanaka_sub_randomf.csv", index=False)

public 0.474

結構精度は出てる．

# ハイパラチューニング

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
cv_params = {
    'n_estimators': [100, 200],
    'max_depth': [4, 8, 16],
    'min_samples_leaf': [1, 5, 10, 50],
}

clf = RandomForestClassifier(criterion='log_loss')

gridcv = GridSearchCV(
    clf, cv_params, cv=3,
    scoring='neg_log_loss', n_jobs=-1
)
gridcv.fit(pd.concat([train_X, valid_X]), pd.concat([train_y, valid_y]))

best_params = gridcv.best_params_
best_score = gridcv.best_score_
print(f'最適パラメータ {best_params}\nスコア {best_score}')

In [ ]:
pred_test = gridcv.predict_proba(test_ft_df)[:, 1]
sub_df = pd.DataFrame({
    "ID": test_df["ID"], 
    "PredictedProb": pred_test
}).to_csv("tanaka_sub_randomf.csv", index=False)

public 0.474

もっと depth 深くすれば上がりそうだけど，ハイパラチューニングに時間かかるし，こんなもんでいいか．